In [ ]:
import utils.plotting as uplt
import radiation_time_step as experiment_runner
from pathlib import Path
import numpy as np
import proplot as pplt
import xarray as xr
from utils.files import OIFSPreprocessor, NEMOPreprocessor

In [ ]:
oifs_preprocessor = OIFSPreprocessor(
    experiment_runner.start_date, np.timedelta64(-7, "h")
)
nemo_preprocessor = NEMOPreprocessor(
    experiment_runner.start_date, np.timedelta64(-7, "h")
)

In [ ]:
plotting_output_directory = Path("plots/radiation_time_step")
plotting_output_directory.mkdir(exist_ok=True)

In [ ]:
exp_ids_long = ["RAL0", "RAL1", "RAL2"]
exp_ids_equal = ["RAE0", "RAE1", "RAE2"]
exp_ids_short = ["RAS0", "RAS1", "RAS2"]
exp_ids_parallel = [exp_ids_long[0], exp_ids_equal[0], exp_ids_short[0]]
exp_ids_atmfirst = [exp_ids_long[1], exp_ids_equal[1], exp_ids_short[1]]
exp_ids_ocefirst = [exp_ids_long[2], exp_ids_equal[2], exp_ids_short[2]]
max_schwarz_iters = experiment_runner.max_iters
setup = "PAPA"

In [ ]:
oifs_progvars_long = [
    xr.open_mfdataset(
        f"{setup}/{exp_id}/progvar.nc", preprocess=oifs_preprocessor.preprocess
    )
    for exp_id in exp_ids_long
]
oifs_diagvars_long = [
    xr.open_mfdataset(
        f"{setup}/{exp_id}/diagvar.nc", preprocess=oifs_preprocessor.preprocess
    )
    for exp_id in exp_ids_long
]
nemo_t_grids_long = [
    xr.open_mfdataset(
        f"{setup}/{exp_id}/{exp_id}*_T.nc", preprocess=nemo_preprocessor.preprocess
    )
    for exp_id in exp_ids_long
]

oifs_progvars_equal = [
    xr.open_mfdataset(
        f"{setup}/{exp_id}/progvar.nc", preprocess=oifs_preprocessor.preprocess
    )
    for exp_id in exp_ids_equal
]
oifs_diagvars_equal = [
    xr.open_mfdataset(
        f"{setup}/{exp_id}/diagvar.nc", preprocess=oifs_preprocessor.preprocess
    )
    for exp_id in exp_ids_equal
]
nemo_t_grids_equal = [
    xr.open_mfdataset(
        f"{setup}/{exp_id}/{exp_id}*_T.nc", preprocess=nemo_preprocessor.preprocess
    )
    for exp_id in exp_ids_equal
]

oifs_progvars_short = [
    xr.open_mfdataset(
        f"{setup}/{exp_id}/progvar.nc", preprocess=oifs_preprocessor.preprocess
    )
    for exp_id in exp_ids_short
]
oifs_diagvars_short = [
    xr.open_mfdataset(
        f"{setup}/{exp_id}/diagvar.nc", preprocess=oifs_preprocessor.preprocess
    )
    for exp_id in exp_ids_short
]
nemo_t_grids_short = [
    xr.open_mfdataset(
        f"{setup}/{exp_id}/{exp_id}*_T.nc", preprocess=nemo_preprocessor.preprocess
    )
    for exp_id in exp_ids_short
]

oifs_progvars_cvg_swz = [
    xr.open_mfdataset(
        f"{setup}/{exp_id}_{max_schwarz_iters}/progvar.nc",
        preprocess=oifs_preprocessor.preprocess,
    )
    for exp_id in exp_ids_parallel
]
oifs_diagvars_cvg_swz = [
    xr.open_mfdataset(
        f"{setup}/{exp_id}_{max_schwarz_iters}/diagvar.nc",
        preprocess=oifs_preprocessor.preprocess,
    )
    for exp_id in exp_ids_parallel
]
nemo_t_grids_cvg_swz = [
    xr.open_mfdataset(
        f"{setup}/{exp_id}_{max_schwarz_iters}/{exp_id}*_T.nc",
        preprocess=nemo_preprocessor.preprocess,
    )
    for exp_id in exp_ids_parallel
]

In [ ]:
colors = ["m", "c", "y"]
labels = ["parallel", "atm-first", "oce-first"]
alpha = 1
linestyles = ["--", ":", "-."]

case = "long"
if case == "long":
    exp_ids = exp_ids_long
    oifs_progvars = oifs_progvars_long
    oifs_diagvars = oifs_diagvars_long
    nemo_t_grids = nemo_t_grids_long
elif case == "equal":
    exp_ids = exp_ids_equal
    oifs_progvars = oifs_progvars_equal
    oifs_diagvars = oifs_diagvars_equal
    nemo_t_grids = nemo_t_grids_equal
elif case == "short":
    exp_ids = exp_ids_short
    oifs_progvars = oifs_progvars_short
    oifs_diagvars = oifs_diagvars_short
    nemo_t_grids = nemo_t_grids_short
else:
    raise ValueError("Case invalid")

fig, axs = pplt.subplots(nrows=3, ncols=1, spany=False)
fig.set_size_inches(15, 10)
fig.suptitle(f"{exp_ids[0]}, {exp_ids[1]}, {exp_ids[2]}", y=0.95, size=14)


uplt.create_atm_temps_plot(axs[0], oifs_progvars, colors, alpha, labels, linestyles)
uplt.create_oce_ssts_plot(axs[1], nemo_t_grids, colors, alpha, labels, linestyles)
uplt.create_atm_ssws_plot(axs[2], oifs_diagvars, colors, alpha, labels, linestyles)
fig.savefig(
    plotting_output_directory / f"naive_coupling_schemes_{case}.pdf",
    bbox_inches="tight",
)

# Naive vs. Schwarz Solution

In [ ]:
case = "long"
if case == "long":
    exp_ids = exp_ids_long
    oifs_progvars = [*oifs_progvars_long, oifs_progvars_cvg_swz[0]]
    oifs_diagvars = [*oifs_diagvars_long, oifs_diagvars_cvg_swz[0]]
    nemo_t_grids = [*nemo_t_grids_long, nemo_t_grids_cvg_swz[0]]
elif case == "equal":
    exp_ids = exp_ids_equal
    oifs_progvars = [*oifs_progvars_equal, oifs_progvars_cvg_swz[1]]
    oifs_diagvars = [*oifs_diagvars_equal, oifs_diagvars_cvg_swz[1]]
    nemo_t_grids = [*nemo_t_grids_equal, nemo_t_grids_cvg_swz[1]]
elif case == "short":
    exp_ids = exp_ids_short
    oifs_progvars = [*oifs_progvars_short, oifs_progvars_cvg_swz[2]]
    oifs_diagvars = [*oifs_diagvars_short, oifs_diagvars_cvg_swz[2]]
    nemo_t_grids = [*nemo_t_grids_short, nemo_t_grids_cvg_swz[2]]
else:
    raise ValueError("Case invalid")

colors = ["m", "c", "y", "k"]
labels = ["parallel", "atmosphere-first", "ocean-first", "converged SWR"]
alpha = 1
linestyles = ["--", ":", "-.", "-"]

fig, axs = pplt.subplots(nrows=3, ncols=1, spany=False)
fig.set_size_inches(15, 10)
fig.suptitle(f"Non-iterative vs. Schwarz ({case} dt_rad)", y=0.95, size=14)

uplt.create_atm_temps_plot(axs[0], oifs_progvars, colors, alpha, labels, linestyles)
uplt.create_oce_ssts_plot(axs[1], nemo_t_grids, colors, alpha, labels, linestyles)
uplt.create_atm_ssws_plot(axs[2], oifs_diagvars, colors, alpha, labels, linestyles)
axs[0].legend(ncols=1)
fig.savefig(
    plotting_output_directory / f"coupling_comparison_{case}.pdf",
    bbox_inches="tight",
)

# Surface Heat Fluxes

In [ ]:
case = "long"
if case == "long":
    exp_ids = exp_ids_long
    oifs_progvars = [*oifs_progvars_long, oifs_progvars_cvg_swz[0]]
    oifs_diagvars = [*oifs_diagvars_long, oifs_diagvars_cvg_swz[0]]
    nemo_t_grids = [*nemo_t_grids_long, nemo_t_grids_cvg_swz[0]]
elif case == "equal":
    exp_ids = exp_ids_equal
    oifs_progvars = [*oifs_progvars_equal, oifs_progvars_cvg_swz[1]]
    oifs_diagvars = [*oifs_diagvars_equal, oifs_diagvars_cvg_swz[1]]
    nemo_t_grids = [*nemo_t_grids_equal, nemo_t_grids_cvg_swz[1]]
elif case == "short":
    exp_ids = exp_ids_short
    oifs_progvars = [*oifs_progvars_short, oifs_progvars_cvg_swz[2]]
    oifs_diagvars = [*oifs_diagvars_short, oifs_diagvars_cvg_swz[2]]
    nemo_t_grids = [*nemo_t_grids_short, nemo_t_grids_cvg_swz[2]]
else:
    raise ValueError("Case invalid")

colors = ["m", "c", "y", "k"]
labels = ["parallel", "atm-first", "oce-first", "converged SWR"]
alpha = 1
linestyles = ["--", ":", "-.", "-"]

fig, axs = pplt.subplots(nrows=2, spany=False)
fig.set_size_inches(10, 8)
fig.suptitle(f"Surface Heat Fluxes ({case} dt_rad)", y=0.95, size=14)

ax = axs[0]
for i in range(len(colors)):
    oifs_diagvar = oifs_diagvars[i]
    color = colors[i]
    label = labels[i]
    linestyle = linestyles[i]
    ax.plot(
        oifs_diagvar.sfc_sen_flx,
        color=color,
        label=label,
        alpha=alpha,
        ls=linestyle,
    )
ax.legend(ncols=1)
ax.format(ylabel=r"Sensible Heat Flux $[W m^{-2}]$", title="", xlabel="Time")

ax = axs[1]

for i in range(len(colors)):
    oifs_diagvar = oifs_diagvars[i]
    color = colors[i]
    label = labels[i]
    linestyle = linestyles[i]
    ax.plot(
        oifs_diagvar.sfc_lat_flx,
        color=color,
        label=label,
        alpha=alpha,
        ls=linestyle,
    )
ax.format(ylabel=r"Latent Heat Flux $[W m^{-2}]$", title="", xlabel="Time")

fig.savefig(
    plotting_output_directory / f"sfc_heat_flux_comparison_{case}.pdf",
    bbox_inches="tight",
)

# Cloud Cover

In [ ]:
case = "long"
if case == "long":
    exp_ids = exp_ids_long
    oifs_progvars = [*oifs_progvars_long, oifs_progvars_cvg_swz[0]]
    oifs_diagvars = [*oifs_diagvars_long, oifs_diagvars_cvg_swz[0]]
    nemo_t_grids = [*nemo_t_grids_long, nemo_t_grids_cvg_swz[0]]
elif case == "equal":
    exp_ids = exp_ids_equal
    oifs_progvars = [*oifs_progvars_equal, oifs_progvars_cvg_swz[1]]
    oifs_diagvars = [*oifs_diagvars_equal, oifs_diagvars_cvg_swz[1]]
    nemo_t_grids = [*nemo_t_grids_equal, nemo_t_grids_cvg_swz[1]]
elif case == "short":
    exp_ids = exp_ids_short
    oifs_progvars = [*oifs_progvars_short, oifs_progvars_cvg_swz[2]]
    oifs_diagvars = [*oifs_diagvars_short, oifs_diagvars_cvg_swz[2]]
    nemo_t_grids = [*nemo_t_grids_short, nemo_t_grids_cvg_swz[2]]
else:
    raise ValueError("Case invalid")

colors = ["m", "c", "y", "k"]
labels = ["parallel", "atm-first", "oce-first", "converged SWR"]
alpha = 1
linestyles = ["--", ":", "-.", "-"]

fig, ax = pplt.subplots()
fig.set_size_inches(10, 6)
fig.suptitle(f"Total Cloud Cover ({case} dt_rad)", y=0.95, size=14)

for i in range(len(colors)):
    oifs_diagvar = oifs_diagvars[i]
    color = colors[i]
    label = labels[i]
    linestyle = linestyles[i]
    ax.plot(
        oifs_diagvar.total_cloud,
        color=color,
        label=label,
        alpha=alpha,
        ls=linestyle,
    )
ax.format(title="", xlabel="Time", ylabel="Cloud Cover")
ax.legend(ncols=1)
fig.savefig(
    plotting_output_directory / f"ccover_{case}.pdf",
    bbox_inches="tight",
)

# Vertical Profiles

## Atmosphere

### Vertical Temperature Profile

In [ ]:
case = "short"
if case == "long":
    exp_ids = exp_ids_long
    oifs_progvars = [*oifs_progvars_long, oifs_progvars_cvg_swz[0]]
    oifs_diagvars = [*oifs_diagvars_long, oifs_diagvars_cvg_swz[0]]
    nemo_t_grids = [*nemo_t_grids_long, nemo_t_grids_cvg_swz[0]]
elif case == "equal":
    exp_ids = exp_ids_equal
    oifs_progvars = [*oifs_progvars_equal, oifs_progvars_cvg_swz[1]]
    oifs_diagvars = [*oifs_diagvars_equal, oifs_diagvars_cvg_swz[1]]
    nemo_t_grids = [*nemo_t_grids_equal, nemo_t_grids_cvg_swz[1]]
elif case == "short":
    exp_ids = exp_ids_short
    oifs_progvars = [*oifs_progvars_short, oifs_progvars_cvg_swz[2]]
    oifs_diagvars = [*oifs_diagvars_short, oifs_diagvars_cvg_swz[2]]
    nemo_t_grids = [*nemo_t_grids_short, nemo_t_grids_cvg_swz[2]]
else:
    raise ValueError("Case invalid")

titles = [
    "Parallel",
    "Sequential Atmosphere-First",
    "Sequential Ocean-First",
    "Converged Schwarz Waveform Relaxation",
]

fig, axs = pplt.subplots(nrows=4, ncols=1)
fig.set_size_inches(15, 13)
fig.suptitle(
    f"Vertical Atmospheric Temperature Profiles in BL ({case} dt_rad)",
    y=1,
    size=14,
)

for i in range(len(oifs_progvars)):
    ax = axs[i]
    ax_title = titles[i]
    oifs_progvar = oifs_progvars[i]
    oifs_progvar = oifs_progvar.assign_coords(
        air_pressure=("nlev", oifs_progvar.pressure_f[0].data / 100)
    )
    oifs_progvar = oifs_progvar.swap_dims({"nlev": "air_pressure"})
    im = ax.contourf(
        oifs_progvar.t[:, 50:] - 273.15,
        levels=10,
        vmin=0,
        vmax=15,
        transpose=True,
        discrete=True,
    )
    ax.format(xlabel="Time", ylabel="Air Pressure [hPa]", title=ax_title)
    ax.invert_yaxis()
fig.colorbar(im, loc="b", title="Temperature [°C]")
fig.savefig(
    plotting_output_directory / f"air_temperature_profile_{case}.pdf",
    bbox_inches="tight",
)

### Vertical Humidity Profile

In [ ]:
case = "long"
if case == "long":
    exp_ids = exp_ids_long
    oifs_progvars = [*oifs_progvars_long, oifs_progvars_cvg_swz[0]]
    oifs_diagvars = [*oifs_diagvars_long, oifs_diagvars_cvg_swz[0]]
    nemo_t_grids = [*nemo_t_grids_long, nemo_t_grids_cvg_swz[0]]
elif case == "equal":
    exp_ids = exp_ids_equal
    oifs_progvars = [*oifs_progvars_equal, oifs_progvars_cvg_swz[1]]
    oifs_diagvars = [*oifs_diagvars_equal, oifs_diagvars_cvg_swz[1]]
    nemo_t_grids = [*nemo_t_grids_equal, nemo_t_grids_cvg_swz[1]]
elif case == "short":
    exp_ids = exp_ids_short
    oifs_progvars = [*oifs_progvars_short, oifs_progvars_cvg_swz[2]]
    oifs_diagvars = [*oifs_diagvars_short, oifs_diagvars_cvg_swz[2]]
    nemo_t_grids = [*nemo_t_grids_short, nemo_t_grids_cvg_swz[2]]
else:
    raise ValueError("Case invalid")

titles = [
    "Parallel",
    "Sequential Atmosphere-First",
    "Sequential Ocean-First",
    "Converged Schwarz Waveform Relaxation",
]

fig, axs = pplt.subplots(nrows=4, ncols=1)
fig.set_size_inches(15, 13)
fig.suptitle(
    f"Vertical Atmospheric Humidity Profiles in BL ({case} dt_rad)",
    y=1,
    size=14,
)

for i in range(len(oifs_progvars)):
    ax = axs[i]
    ax_title = titles[i]
    oifs_progvar = oifs_progvars[i]
    oifs_progvar = oifs_progvar.assign_coords(
        air_pressure=("nlev", oifs_progvar.pressure_f[0].data / 100)
    )
    oifs_progvar = oifs_progvar.swap_dims({"nlev": "air_pressure"})
    im = ax.contourf(
        oifs_progvar.q[:, 50:],
        vmin=0,
        vmax=0.01,
        transpose=True,
        discrete=True,
    )
    ax.set_ylabel("Air Pressure [hPa]")
    ax.invert_yaxis()
    ax.set_title(ax_title)
fig.colorbar(im, loc="b", title="Water Vapor Mixing Ratio [kg/kg]")
fig.savefig(
    plotting_output_directory / f"q_profile_{case}.pdf",
    bbox_inches="tight",
)

## Ocean

### Vertical Temperature Profile

In [ ]:
case = "short"
if case == "long":
    exp_ids = exp_ids_long
    oifs_progvars = [*oifs_progvars_long, oifs_progvars_cvg_swz[0]]
    oifs_diagvars = [*oifs_diagvars_long, oifs_diagvars_cvg_swz[0]]
    nemo_t_grids = [*nemo_t_grids_long, nemo_t_grids_cvg_swz[0]]
elif case == "equal":
    exp_ids = exp_ids_equal
    oifs_progvars = [*oifs_progvars_equal, oifs_progvars_cvg_swz[1]]
    oifs_diagvars = [*oifs_diagvars_equal, oifs_diagvars_cvg_swz[1]]
    nemo_t_grids = [*nemo_t_grids_equal, nemo_t_grids_cvg_swz[1]]
elif case == "short":
    exp_ids = exp_ids_short
    oifs_progvars = [*oifs_progvars_short, oifs_progvars_cvg_swz[2]]
    oifs_diagvars = [*oifs_diagvars_short, oifs_diagvars_cvg_swz[2]]
    nemo_t_grids = [*nemo_t_grids_short, nemo_t_grids_cvg_swz[2]]
else:
    raise ValueError("Case invalid")

titles = [
    "Parallel",
    "Sequential Atmosphere-First",
    "Sequential Ocean-First",
    "Converged Schwarz Waveform Relaxation",
]

fig, axs = pplt.subplots(nrows=4, ncols=1)
fig.set_size_inches(15, 13)
fig.suptitle(
    f"Vertical Ocean Temperature Profiles in Mixed Layer ({case} dt_rad)",
    y=1,
    size=14,
)

for i in range(len(nemo_t_grids)):
    ax = axs[i]
    ax_title = titles[i]
    nemo_t_grid = nemo_t_grids[i]
    im = ax.contourf(
        nemo_t_grid.votemper[:, :10],
        levels=10,
        vmin=10,
        vmax=12,
        discrete=True,
        transpose=True,
    )
    ax.invert_yaxis()
    ax.format(title=ax_title, xlabel="Time", ylabel="Depth [m]")
cax = axs[-1]
fig.colorbar(im, loc="b", title="Temperature [°C]")
fig.savefig(
    plotting_output_directory / f"oce_temperature_profiles_{case}.pdf",
    bbox_inches="tight",
)

# Compare across $\Delta t_\mathrm{rad}$

## Compare Converged Schwarz Solutions in the three cases

In [ ]:
colors = ["m", "c", "y"]
labels = ["long", "equal", "short"]
alpha = 1
linestyles = ["--", ":", "-."]

oifs_progvars = oifs_progvars_cvg_swz
oifs_diagvars = oifs_diagvars_cvg_swz
nemo_t_grids = nemo_t_grids_cvg_swz


fig, axs = pplt.subplots(nrows=3, ncols=1, spany=False)
fig.set_size_inches(15, 10)
fig.suptitle(
    r"Converged Schwarz Solutions for Different $\Delta t_{rad}$", y=0.95, size=14
)


uplt.create_atm_temps_plot(axs[0], oifs_progvars, colors, alpha, labels, linestyles)
uplt.create_oce_ssts_plot(axs[1], nemo_t_grids, colors, alpha, labels, linestyles)
uplt.create_atm_ssws_plot(axs[2], oifs_diagvars, colors, alpha, labels, linestyles)
fig.savefig(
    plotting_output_directory / f"converged_schwarz_solutions_sfcvars.pdf",
    bbox_inches="tight",
)

## Compare Number of Iterations Necessary to Converge

In [ ]:
fig, ax = pplt.subplots()
fig.set_size_inches(10, 6)
x_data = np.arange(1, max_schwarz_iters)

cases = ["3h", "1h", "12min"]
for case_number, exp_id in enumerate(exp_ids_parallel):
    oifs_progvars = [
        xr.open_mfdataset(
            f"PAPA/{exp_id}_{iter}/progvar.nc", preprocess=oifs_preprocessor.preprocess
        )
        for iter in range(1, max_schwarz_iters + 1)
    ]
    oifs_progvar_cvg = oifs_progvars[-1]

    errors = [
        np.linalg.norm((oifs_progvar.t - oifs_progvar_cvg.t).data, ord=np.inf)
        for oifs_progvar in oifs_progvars[:-1]
    ]

    ax.semilogy(x_data, errors, ls="-", marker=".", label=cases[case_number])

ax.format(
    title=r"Errors in 10m Temperature depending on $\Delta t_{rad}$ (OpenIFS, SWR)",
    xlabel="Schwarz Iteration",
    ylabel=r"$||e||_\infty$",
    xlocator=x_data[::2],
    xminorlocator=1,
)
ax.legend()
fig.savefig(
    plotting_output_directory / f"errors_linf_10t_oifs.pdf",
    bbox_inches="tight",
)

In [ ]:
fig, ax = pplt.subplots()
fig.set_size_inches(10, 6)
x_data = np.arange(1, max_schwarz_iters)

cases = ["3h", "1h", "12min"]
for case_number, exp_id in enumerate(exp_ids_parallel):
    nemo_t_grids = [
        xr.open_mfdataset(
            f"PAPA/{exp_id}_{iter}/{exp_id}_*_T.nc",
            preprocess=nemo_preprocessor.preprocess,
        )
        for iter in range(1, max_schwarz_iters + 1)
    ]
    nemo_t_grid_cvg = nemo_t_grids[-1]
    errors = [
        np.linalg.norm(
            (nemo_t_grid.sosstsst - nemo_t_grid_cvg.sosstsst).data,
            ord=np.inf,
        )
        for nemo_t_grid in nemo_t_grids[:-1]
    ]

    ax.semilogy(x_data, errors, ls="-", marker=".", label=cases[case_number])

ax.format(
    title=r"Errors in SST depending on $\Delta t_{rad}$ (NEMO, SWR)",
    xlabel="Schwarz Iteration",
    ylabel=r"$||e||_\infty$",
    xlocator=x_data[::2],
    xminorlocator=1,
)
ax.legend()
fig.savefig(
    plotting_output_directory / f"errors_linf_sst_nemo.pdf",
    bbox_inches="tight",
)

In [ ]:
fig, ax = pplt.subplots()
fig.set_size_inches(10, 6)
x_data = np.arange(1, max_schwarz_iters)

cases = ["3h", "1h", "12min"]
for case_number, exp_id in enumerate(exp_ids_parallel):
    oifs_diagvars = [
        xr.open_mfdataset(
            f"PAPA/{exp_id}_{iter}/diagvar.nc", preprocess=oifs_preprocessor.preprocess
        )
        for iter in range(1, max_schwarz_iters + 1)
    ]
    oifs_diagvar_cvg = oifs_diagvars[-1]

    errors = [
        np.linalg.norm(
            (oifs_diagvar.sfc_sen_flx - oifs_diagvar_cvg.sfc_sen_flx).data, ord=np.inf
        )
        for oifs_diagvar in oifs_diagvars[:-1]
    ]

    ax.semilogy(x_data, errors, ls="-", marker=".", label=cases[case_number])

ax.format(
    title=r"Errors in Surface SH Flux depending on $\Delta t_{rad}$ (OpenIFS, SWR)",
    xlabel="Schwarz Iteration",
    ylabel=r"$||e||_\infty$",
    xlocator=x_data[::2],
    xminorlocator=1,
)
ax.legend()
fig.savefig(
    plotting_output_directory / f"errors_linf_ssh_oifs.pdf",
    bbox_inches="tight",
)

## Compare Difference between Initial Guesses and Converged Solutions

In [ ]:
oifs_progvars = [
    oifs_progvars_long[0],
    oifs_progvars_equal[0],
    oifs_progvars_short[0],
    *oifs_progvars_cvg_swz,
]
oifs_diagvars = [
    oifs_diagvars_long[0],
    oifs_diagvars_equal[0],
    oifs_diagvars_short[0],
    *oifs_diagvars_cvg_swz,
]
nemo_t_grids = [
    nemo_t_grids_long[0],
    nemo_t_grids_equal[0],
    nemo_t_grids_short[0],
    *nemo_t_grids_cvg_swz,
]

colors = [
    "light burgundy",
    "bright sky blue",
    "apple green",
    "burgundy",
    "dark sky blue",
    "emerald",
]
labels = [
    "3h, parallel",
    "1h, parallel",
    "12 min, parallel",
    "3h, converged SWR",
    "1h, converged SWR",
    "12min, converged SWR",
]
alpha = 1
linestyles = ["--", ":", "-.", "--", ":", "-."]

fig, axs = pplt.subplots(nrows=3, ncols=1, spany=False)
fig.set_size_inches(15, 10)
fig.suptitle(
    f"Difference between Initial Guesses and Converged Solutions", y=0.95, size=14
)


uplt.create_atm_temps_plot(axs[0], oifs_progvars, colors, alpha, labels, linestyles)
uplt.create_oce_ssts_plot(axs[1], nemo_t_grids, colors, alpha, labels, linestyles)
uplt.create_atm_ssws_plot(axs[2], oifs_diagvars, colors, alpha, labels, linestyles)
fig.savefig(
    plotting_output_directory / f"first_vs_last_parallel.pdf",
    bbox_inches="tight",
)